In [81]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
from nltk.corpus import stopwords

In [98]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [99]:
df = pd.read_sql_query("SELECT * FROM spark_cleaned", con)

In [100]:
df.head(5)

,index,screen_name,date_time,text,retweeted,lang
0,0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0,en
1,1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0,en
2,2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0,en
3,3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0,en
4,4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0,en


In [101]:
df_drop_index = df.drop(columns = 'index')

In [102]:
df_drop_index.head()

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0,en
4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0,en


In [103]:
len(df_drop_index)

115

In [104]:
# remove duplicates 
df_no_dupes = df_drop_index.drop_duplicates()
df_no_dupes
len(df_no_dupes)

81

In [105]:
# use regex to remove @usernames

df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
#re.sub('@[^\s]+','',Tweet)

<ipython-input-105-59960d4ecb67>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
<ipython-input-105-59960d4ecb67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')


In [106]:
df_no_dupes

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,Yup - I’m the crazy crypto/Tesla guy..,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,I don't believe I ever claimed Tesla had ...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,Media: “TESLA CRASHES INTO BUILDING WHILE ON ...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,Why you have so many followers ?,0,en
4,developrwannab,2021-09-04 13:28:53.000000,Great thanks. Now I got to add Tesla bot 🤔 So...,0,en
...,...,...,...,...,...
110,_bennettm_,2021-09-05 20:42:13.000000,you gotta think bout it. all those sensors...,0,en
111,eu_bike,2021-09-05 20:42:01.000000,"True, however, It could be gett...",0,en
112,ElonPromises,2021-09-05 20:42:01.000000,"""Shareholders could either to sell at 420 or h...",0,en
113,vi_resh,2021-09-05 20:42:00.000000,Dear EV makers\n\nPlease make a universal char...,0,en


In [107]:
# use regex to remove punctuation and emojis 
df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')

<ipython-input-107-0fde4d1c2c14>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')
<ipython-input-107-0fde4d1c2c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')


In [108]:
df_no_dupes

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,Yup Im the crazy cryptoTesla guy,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,I dont believe I ever claimed Tesla had g...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,Media TESLA CRASHES INTO BUILDING WHILE ON AU...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,Why you have so many followers,0,en
4,developrwannab,2021-09-04 13:28:53.000000,Great thanks Now I got to add Tesla bot So t...,0,en
...,...,...,...,...,...
110,_bennettm_,2021-09-05 20:42:13.000000,you gotta think bout it all those sensors ...,0,en
111,eu_bike,2021-09-05 20:42:01.000000,True however It could be gettin...,0,en
112,ElonPromises,2021-09-05 20:42:01.000000,Shareholders could either to sell at 420 or ho...,0,en
113,vi_resh,2021-09-05 20:42:00.000000,Dear EV makers\n\nPlease make a universal char...,0,en


In [113]:
# make all text lowercase

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())

<ipython-input-113-4aad2dac7cc5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())


In [114]:
# remove stop words 
stop = stopwords.words('english')

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

<ipython-input-114-e64ccaa07b0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [115]:
# stop
# list of stop words

In [116]:
df_no_dupes

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,yup im crazy cryptotesla guy,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,dont believe ever claimed tesla great build qu...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,media tesla crashes building autopilot,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,many followers,0,en
4,developrwannab,2021-09-04 13:28:53.000000,great thanks got add tesla bot thats kind deve...,0,en
...,...,...,...,...,...
110,_bennettm_,2021-09-05 20:42:13.000000,gotta think bout sensors airbags body work pai...,0,en
111,eu_bike,2021-09-05 20:42:01.000000,true however could getting point bidding war o...,0,en
112,ElonPromises,2021-09-05 20:42:01.000000,shareholders could either sell 420 hold shares...,0,en
113,vi_resh,2021-09-05 20:42:00.000000,dear ev makers please make universal charger l...,0,en


In [ ]:
# lamb